In [ ]:
import graphviz
from code_data_science import data_table as dt

df = dt.read_csv('../samples/cobol_relationships.csv')

In [ ]:
graphviz.set_jupyter_format('svg')
graphviz.set_default_engine('sfdp')
dot = graphviz.Digraph('cobol-relationships', comment='COBOL relationships')

def style_node(value):
    if value == 'true': 
        return 'red'
    return 'white'

def map_relationship(row):
    dot.node(row['dependent'])
    dot.node(row['dependency'], style='filled', fillcolor=style_node(row['dependencyMissing']))
    dot.edge(row['dependent'], row['dependency'], row['action'])

df.apply(map_relationship, axis=1)

dot